In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
# from pandas.tseries.holiday import 
import holidays

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
from gensim.models.wrappers import FastText
fmodel = FastText.load_fasttext_format('wiki.en/wiki.en.bin')

In [243]:
df_train = pd.read_csv('Dataset.csv')
df_train.sort_values(by='Date',inplace=True)
df_train.head(2)

,Date,Market,Keyword,Average.Position,CPC,Clicks,CTR,Impressions,Cost
0,20120524,US-Market,secure online back up,0.0,0.0,0.0,0.00%,0.0,0.0
305,20120524,US-Market,donate to haiti,0.0,0.0,0.0,0.00%,0.0,0.0


In [244]:
df_test = pd.read_csv('Evaluate.csv')
df_test.head(2)

,Date,Market,Keyword,Average.Position,CPC,CTR,Impressions,Cost
0,20130214,US-Market,cheap insurance life term,0.0,0.00,0.00%,0,0.00
1,20130214,US-Market,home loans refinancing,1.0,7.63,3.80%,445,130.41


In [245]:
def get_model2(data,data_test):
    data['Date'] = data['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='ignore'))
    data_test['Date'] = data_test['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='ignore'))
    
    date_data = data['Date']

    date = pd.DataFrame({'year':date_data.dt.year,
                         'month':date_data.dt.month,
                        'week':date_data.dt.week,
                        'dayofweek':date_data.dt.dayofweek, #date_data.dt.weekday
                        'dayofyear':date_data.dt.dayofyear})

    date_data_test = data_test['Date']

    date_test = pd.DataFrame({'year':date_data_test.dt.year,
                         'month':date_data_test.dt.month,
                        'week':date_data_test.dt.week,
                        'dayofweek':date_data_test.dt.dayofweek, #date_data.dt.weekday
                        'dayofyear':date_data_test.dt.dayofyear})
    
    train_data = pd.concat([data,date],axis=1)
    test_data = pd.concat([data_test,date_test],axis=1)
    train_data.dropna(inplace=True)
    
    train_data['Market'] = train_data['Market'].astype("category")
    train_data['Market'] = train_data['Market'].cat.codes
    test_data['Market'] = test_data['Market'].astype("category")
    test_data['Market'] = test_data['Market'].cat.codes
    
    train_data['CTR']=train_data['CTR'].str.rstrip('%').astype('float')
    test_data['CTR']=test_data['CTR'].str.rstrip('%').astype('float')
    
    train = train_data[train_data['year']!=2013]
    val = train_data[train_data['year']==2013]
    
    train.drop(['Date','year'],axis=1,inplace=True)
    val.drop(['Date','year'],axis=1,inplace=True)
    test_data.drop(['Date','year'],axis=1,inplace=True)
    
    X_train = train.drop('Clicks',axis=1)
    Y_train = train['Clicks']
    X_valid = val.drop('Clicks',axis=1)
    Y_valid = val['Clicks']
    return X_train,Y_train,X_valid,Y_valid,test_data
    

In [ ]:
def get_model1(data,data_test):
    data['Date'] = data['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='ignore'))
    data_test['Date'] = data_test['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='ignore'))
    
    date_data = data['Date']
    date = pd.DataFrame({'year':date_data.dt.year})

    date_data_test = data_test['Date']
    date_test = pd.DataFrame({'year':date_data_test.dt.year})
    
    train_data = pd.concat([data,date],axis=1)
    test_data = pd.concat([data_test,date_test],axis=1)
    train_data.dropna(inplace=True)
    
    train_data['Market'] = train_data['Market'].astype("category")
    train_data['Market'] = train_data['Market'].cat.codes
    test_data['Market'] = test_data['Market'].astype("category")
    test_data['Market'] = test_data['Market'].cat.codes
    
    train_data['CTR']=train_data['CTR'].str.rstrip('%').astype('float')
    test_data['CTR']=test_data['CTR'].str.rstrip('%').astype('float')
    
    train = train_data[train_data['year']!=2013]
    val = train_data[train_data['year']==2013]
    
    train.drop(['Date','year'],axis=1,inplace=True)
    val.drop(['Date','year'],axis=1,inplace=True)
    test_data.drop(['Date','year'],axis=1,inplace=True)
    
    X_train = train.drop('Clicks',axis=1)
    Y_train = train['Clicks']
    X_valid = val.drop('Clicks',axis=1)
    Y_valid = val['Clicks']
    
    return X_train,Y_train,X_valid,Y_valid,test_data


In [246]:
def get_model3(data,data_test):
    data['Date'] = data['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='ignore'))
    data_test['Date'] = data_test['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='ignore'))
    
    date_data = data['Date']
    date = pd.DataFrame({'year':date_data.dt.year})

    date_data_test = data_test['Date']
    date_test = pd.DataFrame({'year':date_data_test.dt.year})
    
    train_data = pd.concat([data,date],axis=1)
    test_data = pd.concat([data_test,date_test],axis=1)
    train_data.dropna(inplace=True)
    
    train_data['CTR']=train_data['CTR'].str.rstrip('%').astype('float')
    test_data['CTR']=test_data['CTR'].str.rstrip('%').astype('float')
    train_data['Market'] = train_data['Market'].astype("category")
    train_data['Market'] = train_data['Market'].cat.codes
    test_data['Market'] = test_data['Market'].astype("category")
    test_data['Market'] = test_data['Market'].cat.codes
    
    train = train_data[train_data['year']!=2013]
    val = train_data[train_data['year']==2013]
    
    train.drop(['Date','year','Average.Position','CTR'],axis=1,inplace=True)
    val.drop(['Date','year','Average.Position','CTR'],axis=1,inplace=True)
    test_data.drop(['Date','year','Average.Position','CTR'],axis=1,inplace=True)
    
    X_train = train.drop('Clicks',axis=1)
    Y_train = train['Clicks']
    X_valid = val.drop('Clicks',axis=1)
    Y_valid = val['Clicks']
    
    return X_train,Y_train,X_valid,Y_valid,test_data

In [247]:
def get_model4(data,data_test):
    data['Date'] = data['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='ignore'))
    data_test['Date'] = data_test['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d', errors='ignore'))
    
    date_data = data['Date']
    date = pd.DataFrame({'year':date_data.dt.year})

    date_data_test = data_test['Date']
    date_test = pd.DataFrame({'year':date_data_test.dt.year})
    
    train_data = pd.concat([data,date],axis=1)
    test_data = pd.concat([data_test,date_test],axis=1)
    train_data.dropna(inplace=True)
    
    train_data['CTR']=train_data['CTR'].str.rstrip('%').astype('float')
    test_data['CTR']=test_data['CTR'].str.rstrip('%').astype('float')
    
    train_2012 = train_data[train_data['year']!=2013]
    train_2013 = train_data[train_data['year']==2013]
    uk_2012 = train_2012[train_2012['Market']=='UK-Market']
    us_2012 = train_2012[train_2012['Market']=='US-Market']
    uk_2013 = train_2013[train_2013['Market']=='UK-Market']
    us_2013 = train_2013[train_2013['Market']=='US-Market']
    uk_2012['holidays']=uk_2012['Date'].isin(holidays.UnitedKingdom(years = 2012))
    uk_2013['holidays']=uk_2013['Date'].isin(holidays.UnitedKingdom(years = 2013))
    us_2012['holidays']=us_2012['Date'].isin(holidays.UnitedStates(years = 2012))
    us_2013['holidays']=us_2013['Date'].isin(holidays.UnitedStates(years = 2013))

    uk = test_data[test_data['Market']=='UK-Market']
    us = test_data[test_data['Market']=='US-Market']
    uk['holidays']=uk['Date'].isin(holidays.UnitedKingdom(years = 2013))
    us['holidays']=us['Date'].isin(holidays.UnitedStates(years = 2013))
    
    train_data = pd.concat([uk_2012,us_2012,uk_2013,us_2013],axis=0)
    test_data = pd.concat([uk,us],axis=0)
    
    train_data['Market'] = train_data['Market'].astype("category")
    train_data['Market'] = train_data['Market'].cat.codes
    train_data['holidays'] = train_data['holidays'].astype("category")
    train_data['holidays'] = train_data['holidays'].cat.codes
    test_data['Market'] = test_data['Market'].astype("category")
    test_data['Market'] = test_data['Market'].cat.codes
    test_data['holidays'] = test_data['holidays'].astype("category")
    test_data['holidays'] = test_data['holidays'].cat.codes
    
    train = train_data[train_data['year']!=2013]
    val = train_data[train_data['year']==2013]
    
    train.drop(['Date','year','Average.Position','CTR'],axis=1,inplace=True)
    val.drop(['Date','year','Average.Position','CTR'],axis=1,inplace=True)
    test_data.drop(['Date','year','Average.Position','CTR'],axis=1,inplace=True)
    
    X_train = train.drop('Clicks',axis=1)
    Y_train = train['Clicks']
    X_valid = val.drop('Clicks',axis=1)
    Y_valid = val['Clicks']
    
    return X_train,Y_train,X_valid,Y_valid,test_data

In [216]:
X_train,Y_train,X_valid,Y_valid,test = get_model2(df_train,df_test)

In [217]:
X_train,Y_train,X_valid,Y_valid,test = get_model1(df_train,df_test)

In [249]:
X_train,Y_train,X_valid,Y_valid,test = get_model3(df_train,df_test)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [218]:
X_train,Y_train,X_valid,Y_valid,test = get_model4(df_train,df_test)

In [250]:
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)
print(test.shape)

(285935, 5)
(285935,)
(66605, 5)
(66605,)
(400, 5)


In [251]:
X_train.head(2)

,Market,Keyword,CPC,Impressions,Cost
0,1,secure online back up,0.0,0.0,0.0
305,1,donate to haiti,0.0,0.0,0.0


In [252]:
#Preprocessing - removing unwanted characters, tokenization, stop-word removal
def clean_data(txts):
    x = re.sub("[^a-zA-Z]", " ",txts) 
    x = x.lower().split()                             
    stops = set(stopwords.words("english"))                  
    words = [w for w in x if not w in stops] 
    
    return( " ".join(words))

In [253]:
X_train['Keyword'] = X_train['Keyword'].apply(lambda x: clean_data(x))
X_valid['Keyword']= X_valid['Keyword'].apply(lambda x: clean_data(x))
test['Keyword']= test['Keyword'].apply(lambda x: clean_data(x))

In [254]:
train_words = X_train['Keyword'].tolist()
val_words = X_valid['Keyword'].tolist()
test_words = test['Keyword'].tolist()

#======================================================================
X_train_words= np.zeros((len(train_words),300)) 

for i in range(len(train_words)):
    words = word_tokenize(train_words[i])
    emb = [fmodel.wv[w] for w in words] 
    X_train_words[i] = np.mean(emb, axis=0)
    
df_train_1= pd.DataFrame(X_train_words)
df_train_1.fillna(value=0,inplace=True)

X_train_words = np.array(df_train_1)
print(X_train_words.shape)

#======================================================================

X_val_words = np.zeros((len(val_words),300))

for i in range(len(val_words)):
    words = word_tokenize(val_words[i])
    emb = [fmodel.wv[w] for w in words]
    X_val_words[i] = np.mean(emb, axis=0)
    
df_test_1 = pd.DataFrame(X_val_words)
df_test_1.fillna(value=0,inplace=True)

X_val_words = np.array(df_test_1)
print(X_val_words.shape)

#======================================================================

X_test_words = np.zeros((len(test_words),300))

for i in range(len(test_words)):
    words = word_tokenize(test_words[i])
    emb = [fmodel.wv[w] for w in words]
    X_test_words[i] = np.mean(emb, axis=0)
    
df_test_2 = pd.DataFrame(X_test_words)
df_test_2.fillna(value=0,inplace=True)

X_test_words = np.array(df_test_2)
print(X_test_words.shape)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(285935, 300)
(66605, 300)
(400, 300)


In [255]:
X_train.drop(['Keyword'],inplace=True,axis=1)
X_valid.drop(['Keyword'],inplace=True,axis=1)
test.drop(['Keyword'],inplace=True,axis=1)

In [256]:
X_train.head(2)

,Market,CPC,Impressions,Cost
0,1,0.0,0.0,0.0
305,1,0.0,0.0,0.0


In [257]:
X_train.values.max(),X_train.values.min()

(3666042.0, 0.0)

In [258]:
test.head(2)

,Market,CPC,Impressions,Cost
0,1,0.00,0,0.00
1,1,7.63,445,130.41


In [259]:
X_train = (X_train.values - X_train.values.min())/(X_train.values.max()-X_train.values.min())
X_valid = (X_valid.values - X_valid.values.min())/(X_valid.values.max()-X_valid.values.min())
test = (test.values - test.values.min())/(test.values.max()-test.values.min())

In [260]:
x_train = np.hstack((X_train_words,X_train))
x_valid = np.hstack((X_val_words,X_valid))
x_test = np.hstack((X_test_words,test))

In [261]:
x_train.shape,x_valid.shape,Y_train.shape,Y_valid.shape,x_test.shape

((285935, 304), (66605, 304), (285935,), (66605,), (400, 304))

In [262]:
x_train.min(),x_train.max()

(-1.3244595527648926, 1.2421350479125977)

### Random Forest Model

In [263]:
from sklearn.ensemble import RandomForestRegressor
classifier = RandomForestRegressor() 
classifier.fit(x_train,Y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [264]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
model_score = classifier.score(x_train,Y_train)
print('coefficient of determination R^2 of the prediction.: ',model_score)
y_predicted = classifier.predict(x_valid)

# The mean squared error
print("Mean squared error: %.2f"% mean_squared_error(Y_valid, y_predicted))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(Y_valid, y_predicted))
print('mean_absolute_error: %.2f' % mean_absolute_error(Y_valid, y_predicted))

coefficient of determination R^2 of the prediction.:  0.9982325987642072
Mean squared error: 121619.00
Test Variance score: 0.92
mean_absolute_error: 42.45


In [215]:
y_pred_1 = classifier.predict(x_test)
y_pred_1[y_pred_1<0] = 0
sub = pd.read_csv('Evaluate.csv')
sub['clicks'] = y_pred_1.astype(float)
sub.to_csv('Randomforest_4.csv',index=False) 
sub.head()

,Date,Market,Keyword,Average.Position,CPC,CTR,Impressions,Cost,clicks
0,20130214,US-Market,cheap insurance life term,0.0,0.00,0.00%,0,0.00,4.834382e-08
1,20130214,US-Market,home loans refinancing,1.0,7.63,3.80%,445,130.41,4.834382e-08
2,20130214,US-Market,consolidate college loans,0.0,0.00,0.00%,0,0.00,4.834382e-08
3,20130214,US-Market,cheap life insurance,1.0,58.88,3.30%,931,1806.63,4.746000e+00
4,20130214,US-Market,vioxx law suit,0.0,0.00,0.00%,0,0.00,4.834382e-08


### XGboost Model

In [150]:
import xgboost

best_xgb_model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.05,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=30,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
best_xgb_model.fit(x_train,Y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[18:03:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1.5, missing=None, n_estimators=30,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.6, verbosity=1)

In [151]:
pred_1 = best_xgb_model.predict(x_valid)

In [152]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
model_score = best_xgb_model.score(x_train,Y_train)
print('coefficient of determination R^2 of the prediction.: ',model_score)
y_predicted = best_xgb_model.predict(x_valid)

# The mean squared error
print("Mean squared error: %.2f"% mean_squared_error(Y_valid, y_predicted))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(Y_valid, y_predicted))
print('mean_absolute_error: %.2f' % mean_absolute_error(Y_valid, y_predicted))

coefficient of determination R^2 of the prediction.:  0.7978348112189357
Mean squared error: 135758.68
Test Variance score: 0.91
mean_absolute_error: 103.53


In [153]:
y_pred_1 = best_xgb_model.predict(x_test)
y_pred_1[y_pred_1<0] = 0
sub = pd.read_csv('Evaluate.csv')
sub['clicks'] = y_pred_1.astype(float)
sub.to_csv('xgboost_3.csv',index=False) 
sub.head()

,Date,Market,Keyword,Average.Position,CPC,CTR,Impressions,Cost,clicks
0,20130214,US-Market,cheap insurance life term,0.0,0.00,0.00%,0,0.00,47.276573
1,20130214,US-Market,home loans refinancing,1.0,7.63,3.80%,445,130.41,47.276573
2,20130214,US-Market,consolidate college loans,0.0,0.00,0.00%,0,0.00,47.276573
3,20130214,US-Market,cheap life insurance,1.0,58.88,3.30%,931,1806.63,146.617508
4,20130214,US-Market,vioxx law suit,0.0,0.00,0.00%,0,0.00,47.276573


### 1-D CNN model

In [53]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import Normalizer

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution1D, MaxPooling1D,LSTM,GRU
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import keras

import h5py
from keras.models import Model
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [33]:
X_train1 = x_train.reshape(-1,302,1)
X_valid1 = x_valid.reshape(-1,302,1)

In [35]:
print(X_train1.shape)
print(Y_train.shape)
print(X_valid1.shape)
print(Y_valid.shape)

(285935, 302, 1)
(285935,)
(66605, 302, 1)
(66605,)


In [36]:
model=Sequential()
model.add(Convolution1D(64,3,border_mode='same',activation='relu',input_shape=(302, 1)))
model.add(MaxPooling1D(pool_length=(2)))
model.add(Flatten())
model.add(Dense(128,kernel_initializer='normal',activation='relu'))
model.add(Dense(64,kernel_initializer='normal',activation='relu'))
model.add(Dense(128,kernel_initializer='normal',activation='relu'))
model.add(Dense(32,kernel_initializer='normal',activation='relu'))
# model.add(Dropout(0.2))  # to reduce overfitting, some neuron connections will be cut out
model.add(Dense(1,kernel_initializer='normal',activation='linear'))
model.summary()
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 302, 64)           256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 151, 64)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9664)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1237120   
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dens

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(302, 1), padding="same")`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [65]:
model.fit(X_train1,Y_train, epochs=10, batch_size=100)

Epoch 1/10
285935/285935 [==============================] - 14s 48us/step - loss: 75.7979 - mean_absolute_error: 75.7979
Epoch 2/10
285935/285935 [==============================] - 14s 48us/step - loss: 75.8981 - mean_absolute_error: 75.8981
Epoch 3/10
285935/285935 [==============================] - 14s 48us/step - loss: 75.5127 - mean_absolute_error: 75.5127
Epoch 4/10
285935/285935 [==============================] - 14s 49us/step - loss: 75.0671 - mean_absolute_error: 75.0671
Epoch 5/10
285935/285935 [==============================] - 14s 47us/step - loss: 75.0344 - mean_absolute_error: 75.0344
Epoch 6/10
285935/285935 [==============================] - 14s 47us/step - loss: 75.0878 - mean_absolute_error: 75.0878
Epoch 7/10
285935/285935 [==============================] - 14s 48us/step - loss: 75.4118 - mean_absolute_error: 75.4118
Epoch 8/10
285935/285935 [==============================] - 14s 48us/step - loss: 74.4533 - mean_absolute_error: 74.4533
Epoch 9/10
285935/285935 [======

In [66]:
model.save_weights('results/model10.h5')

In [ ]:
model.load_weights('results/model10.h5')

In [67]:
y_predicted = model.predict(X_valid1)

In [69]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print("Mean squared error: %.2f"% mean_squared_error(Y_valid, y_predicted))
# Explained variance score: 1 is perfect prediction
print('R2 score: %.2f' % r2_score(Y_valid, y_predicted))
print('mean_absolute_error: %.2f' % mean_absolute_error(Y_valid, y_predicted))

Mean squared error: 183220.71
Test Variance score: 0.88
mean_absolute_error: 61.83


### GRU Model

In [154]:
x_train_c = np.reshape(x_train.copy(),((x_train.shape[0],1,x_train.shape[1])))
x_val_c = np.reshape(x_valid.copy(),((x_valid.shape[0],1,x_valid.shape[1])))
x_test_c = np.reshape(x_test.copy(),((x_test.shape[0],1,x_test.shape[1])))
x_train_c.shape,x_val_c.shape

((285935, 1, 304), (66605, 1, 304))

In [155]:
model = Sequential()
model.add(GRU(32, input_shape=(None, x_train_c.shape[2])))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.summary()
model.compile(optimizer=keras.optimizers.Adam(lr=0.005), loss="mae",metrics=['mean_absolute_error'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 32)                32352     
_________________________________________________________________
dense_9 (Dense)              (None, 10)                330       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 11        
Total params: 32,693
Trainable params: 32,693
Non-trainable params: 0
_________________________________________________________________


In [166]:
model.fit(x_train_c,Y_train,epochs=10,batch_size=32,verbose=1,validation_data=(x_val_c, Y_valid))

Train on 285935 samples, validate on 66605 samples
Epoch 1/10
285935/285935 [==============================] - 32s 113us/step - loss: 107.4018 - mean_absolute_error: 107.4018 - val_loss: 99.2431 - val_mean_absolute_error: 99.2431
Epoch 2/10
285935/285935 [==============================] - 32s 113us/step - loss: 107.3911 - mean_absolute_error: 107.3911 - val_loss: 100.2571 - val_mean_absolute_error: 100.2571
Epoch 3/10
285935/285935 [==============================] - 33s 114us/step - loss: 108.2500 - mean_absolute_error: 108.2500 - val_loss: 98.4036 - val_mean_absolute_error: 98.4036
Epoch 4/10
285935/285935 [==============================] - 33s 114us/step - loss: 107.4154 - mean_absolute_error: 107.4154 - val_loss: 107.6675 - val_mean_absolute_error: 107.6675
Epoch 5/10
285935/285935 [==============================] - 33s 114us/step - loss: 105.8633 - mean_absolute_error: 105.8633 - val_loss: 88.6431 - val_mean_absolute_error: 88.6431
Epoch 6/10
285935/285935 [========================

In [162]:
model.save_weights('GRU_model_3_20.h5')

In [120]:
model.load_weights('GRU_model_1_20.h5')

In [163]:
y_predicted = model.predict(x_val_c)

In [164]:
print("Mean squared error: %.2f"% mean_squared_error(Y_valid, y_predicted))
# Explained variance score: 1 is perfect prediction
print('R2 score: %.2f' % r2_score(Y_valid, y_predicted))
print('mean_absolute_error: %.2f' % mean_absolute_error(Y_valid, y_predicted))

Mean squared error: 218947.84
R2 score: 0.85
mean_absolute_error: 85.11


In [165]:
y_pred_1 = model.predict(x_test_c)
y_pred_1[y_pred_1<0] = 0
sub = pd.read_csv('Evaluate.csv')
sub['clicks'] = y_pred_1.astype(float)
sub.to_csv('GRU_3_20epochs.csv',index=False) 
sub.head()

,Date,Market,Keyword,Average.Position,CPC,CTR,Impressions,Cost,clicks
0,20130214,US-Market,cheap insurance life term,0.0,0.00,0.00%,0,0.00,0.000000
1,20130214,US-Market,home loans refinancing,1.0,7.63,3.80%,445,130.41,94.773216
2,20130214,US-Market,consolidate college loans,0.0,0.00,0.00%,0,0.00,2.120856
3,20130214,US-Market,cheap life insurance,1.0,58.88,3.30%,931,1806.63,233.569427
4,20130214,US-Market,vioxx law suit,0.0,0.00,0.00%,0,0.00,0.032273
